<a href="https://colab.research.google.com/github/JOSEPHINEGEND/Moringa_Data_Science_Prep_W2_Independent_Project_2019_06_Josephine_Wanjiku_SQL_Notebook./blob/master/Moringa_Data_Science_Prep_W2_Independent_Project_2019_06_Josephine_Wanjiku_SQL_Notebook_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# First, we load the sql extention into our environment

%load_ext sql

# we then connect our memory sqlite database 

%sql sqlite://

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @None'

In [0]:
# Importing pandas 
import pandas as pd

In [0]:
# Loading the Grand electors by state dataset

with open ('GrandElectors_by_state.csv', 'r') as f:
  GrandElectors = pd.read_csv(f, index_col=0, encoding='utf-8') 

In [0]:
# Removing any similar table existing in our database
# then persist GrandElectors table
# preview GrandElectors table


%sql DROP TABLE IF EXISTS GrandElectors;
%sql PERSIST GrandElectors;
%sql SELECT * FROM GrandElectors limit 5 

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55


In [0]:
# Loading the population by state dataset
with open ('Population_by_state.csv', 'r') as f:
  Population = pd.read_csv(f, index_col=0, encoding='utf-8') 

In [0]:
# Removing any similar table existing in our database
# then persist Population table
# preview Population table


%sql DROP TABLE IF EXISTS Population;
%sql PERSIST Population;
%sql SELECT * FROM Population limit 5 

 * sqlite://
Done.
 * sqlite://
 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995


In [0]:
# Converting ALL the States from GenderElectors to UPPER case
%%sql
UPDATE GrandElectors
SET State = UPPER(State)

 * sqlite://
51 rows affected.


[]

In [0]:
# preview to see changes made on GrandElectors table
%%sql
SELECT * from GrandElectors limit 6

 * sqlite://
Done.


State,GrandElectors
ALABAMA,9
ALASKA,3
ARIZONA,11
ARKANSAS,6
CALIFORNIA,55
COLORADO,9


In [0]:
# Joining the two tables on the state key
# by creating another table called 'MERGEDTABLE'
# then using INNER JOIN function to join them
%%sql
CREATE TABLE MERGEDTABLE AS SELECT State, Population, GrandElectors 
FROM (SELECT * FROM Population
INNER JOIN GrandElectors ON 
Population.State = GrandElectors.State)

 * sqlite://
Done.


[]

In [0]:
# to preview the merged table
%%sql
SELECT * FROM MERGEDTABLE limit 5

 * sqlite://
Done.


State,Population,GrandElectors
CALIFORNIA,39144818,55
TEXAS,27469114,38
FLORIDA,20271272,29
NEW YORK,19795791,29
ILLINOIS,12859995,20


In [0]:
# To change the name of the "District of Columbia" state to its short version "DC"
%%sql
UPDATE MERGEDTABLE 
SET State = "DC"
WHERE State = "DISTRICT OF COLUMBIA"


 * sqlite://
1 rows affected.


[]

In [0]:
# computing the ratio between the number of grand electors and the population
# creating a new column to display that ratio
%%sql
ALTER TABLE MERGEDTABLE ADD RatioGP float(20)


 * sqlite://
Done.


[]

In [0]:
# to preview the table with added column
%%sql
SELECT * FROM MERGEDTABLE limit 5

 * sqlite://
Done.


State,Population,GrandElectors,RatioGP
CALIFORNIA,39144818,55,None
TEXAS,27469114,38,None
FLORIDA,20271272,29,None
NEW YORK,19795791,29,None
ILLINOIS,12859995,20,None


In [0]:
# Updating the RatioGP column with
# calculated ratio which is Population/GrandElectors
# then order the states by decreasing ratio 

# this will make our priority list.

%%sql
UPDATE MERGEDTABLE
SET "RatioGP" = Population/GrandElectors;
SELECT * FROM MERGEDTABLE 
ORDER BY RatioGP ASC limit 6;

 * sqlite://
51 rows affected.
Done.


State,Population,GrandElectors,RatioGP
WYOMING,586107,3,195369.0
VERMONT,626042,3,208680.0
DC,672228,3,224076.0
ALASKA,738432,3,246144.0
NORTH DAKOTA,756927,3,252309.0
RHODE ISLAND,1056298,4,264074.0


In [0]:
# Computing the running total of Grand Electors in the sorted list.

%%sql
SELECT SUM(GrandElectors) from MERGEDTABLE

 * sqlite://
Done.


SUM(GrandElectors)
538


In [0]:
# Independently, to compute the half of the total of Grand Electors overall (in the whole country):
# This is the threshold we need to reach for winning the presidential election.

%%sql
SELECT SUM(GrandElectors)/2 from MERGEDTABLE


 * sqlite://
Done.


SUM(GrandElectors)/2
269


In [0]:
# To filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold.
# (the other states can be ignored). That is our target list.
# Hint: You can do that in 2 steps:
# Select all the states for which the running total is below or equal to the threshold.
# Add the first state for which the running total is larger than the threshold.


%%sql
SELECT a1.State, a1.GrandElectors, SUM(a2.GrandElectors) Running_Total
FROM MERGEDTABLE a1, MERGEDTABLE a2
WHERE a1.GrandElectors <= a2.GrandElectors or (a1.GrandElectors=a2.GrandElectors and a1.State = a2.State)
GROUP BY a1.State, a1.GrandElectors
ORDER BY a1.GrandElectors DESC, a1.State DESC;

 * sqlite://
Done.


State,GrandElectors,Running_Total
CALIFORNIA,55,55
TEXAS,38,93
NEW YORK,29,151
FLORIDA,29,151
PENNSYLVANIA,20,191
ILLINOIS,20,191
OHIO,18,209
MICHIGAN,16,241
GEORGIA,16,241
NORTH CAROLINA,15,256


In [0]:
# That is our target list
%%sql
SELECT a1.State, a1.GrandElectors, SUM(a2.GrandElectors) Running_Total
FROM MERGEDTABLE a1, MERGEDTABLE a2
WHERE a1.GrandElectors <= a2.GrandElectors or (a1.GrandElectors=a2.GrandElectors and a1.State = a2.State)
GROUP BY a1.State, a1.GrandElectors
ORDER BY a1.GrandElectors DESC, a1.State DESC limit 11

 * sqlite://
Done.


State,GrandElectors,Running_Total
CALIFORNIA,55,55
TEXAS,38,93
NEW YORK,29,151
FLORIDA,29,151
PENNSYLVANIA,20,191
ILLINOIS,20,191
OHIO,18,209
MICHIGAN,16,241
GEORGIA,16,241
NORTH CAROLINA,15,256
